In [ ]:
# Ознайомитись з додаванням власних прикладів до моделей spaCy та компонентом для класифікації текстів.
# (Розпізнавання сутностей та класифікація текстів за допомогою textCat)
import spacy
import json
from spacy.training.example import Example
from spacy.util import minibatch
import random

# Зчитування даних із JSON-файлу
with open("1.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Підготовка у форматі (text, {"entities": [(start, end, label)]})
TRAIN_DATA = [(item["text"], {"entities": [tuple(ent) for ent in item["entities"]]}) for item in raw_data]

# Створення нової пустої моделі (англ)
nlp = spacy.blank("en")

# Додаємо NER-пайплайн (Named Entity Recognizer - модель розпізнавання іменованих сутностей)
if "ner" not in nlp.pipe_names: # перевірка імен сутностей в конвеєрі 
    ner = nlp.add_pipe("ner") #додаємо розпізнавальник до конвеєра
else:
    ner = nlp.get_pipe("ner") #дістаємо модель з конвеєру

# Додаємо нову мітку сутності
LABEL = "DISEASE"
ner.add_label(LABEL)


# Ініціалізація параметрів моделі
optimizer = nlp.initialize()

# Навчання моделі
for epoch in range(15):
    losses = {}
    random.shuffle(TRAIN_DATA)
    batches = minibatch(TRAIN_DATA, size=2) # для оновлення вагів моделі після обробки 1-го батчу
    for batch in batches:
        examples = []
        for text, annotations in batch:
            example = Example.from_dict(nlp.make_doc(text), annotations) # Приклад для навчання з пари текст-анотація
            examples.append(example)
            #nlp.update([example], sgd=optimizer, drop=0.25, losses=losses) # Додавання нових прикладів в модель з вимкненням drop% нейронів для запобігання перенавчанню та записом втрат
        nlp.update(examples, sgd=optimizer, drop=0.25, losses=losses)
    print(f"Epoch {epoch+1}: {losses}")

# Збереження моделі
#ner = nlp.get_pipe("ner")
#ner.to_disk("medical_ner_model")
nlp.to_disk("medical_ner_model")
print("Модель збережено в папку 'medical_ner_model'")

# Тестові приклади для прогнозування
test_texts = [
    "The patient was diagnosed with pneumonia.",
    "He suffers from chronic asthma.",
    "She was treated for breast cancer.",
    "The doctor confirmed a diagnosis of diabetes.",
    "They are treating him for hypertension.",
    "This is just for a test"
]

# nlp = spacy.load("medical_ner_model")

print("\nПрогнозовані сутності:")
for text in test_texts:
    doc = nlp(text)
    print(f"\nТекст: {text}")
    for ent in doc.ents:
        print(f" - Сутність: {ent.text}, Мітка: {ent.label_}, Початок: {ent.start_char}, Кінець: {ent.end_char}")


Epoch 1: {'ner': 43.953558921813965}
Epoch 2: {'ner': 9.589602072468338}
Epoch 3: {'ner': 9.210063129169928}
Epoch 4: {'ner': 6.691108042699546}
Epoch 5: {'ner': 2.5227867507614907}
Epoch 6: {'ner': 1.2956727009249496}
Epoch 7: {'ner': 0.923140477453687}
Epoch 8: {'ner': 1.75360219311091}
Epoch 9: {'ner': 3.3206068893509024}
Epoch 10: {'ner': 1.0034120495169685}
Epoch 11: {'ner': 0.40066619597927194}
Epoch 12: {'ner': 0.00032825979292121823}
Epoch 13: {'ner': 0.0005531579285755798}
Epoch 14: {'ner': 1.3040752129192714e-05}
Epoch 15: {'ner': 1.0126008759119838e-05}
Модель збережено в папку 'medical_ner_model'

Прогнозовані сутності:

Текст: The patient was diagnosed with pneumonia.
 - Сутність: pneumonia, Мітка: DISEASE, Початок: 31, Кінець: 40

Текст: He suffers from chronic asthma.
 - Сутність: chronic asthma, Мітка: DISEASE, Початок: 16, Кінець: 30

Текст: She was treated for breast cancer.
 - Сутність: cancer, Мітка: DISEASE, Початок: 27, Кінець: 33

Текст: The doctor confirmed a di

In [ ]:
# Ознайомитись з додаванням власних прикладів до моделей spaCy та компонентом для класифікації текстів.
# (Розпізнавання сутностей та класифікація текстів за допомогою textCat)
import json
import random
import spacy
from spacy.training.example import Example
from spacy.util import minibatch

# Завантаження даних
with open("2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Підготовка тренувальних прикладів
train_data = []
for item in data:
    utterance = item["utterance"] # Висловлювання (текст/повідомлення)
    intent = item["intent"] # Намір повідомлення / категорія (клас)
    train_data.append((utterance, {"cats": {intent: 1.0}})) # Cтворємо тренувальний словник категорій для класифікації, де відповідний intent має ймовірність 1.0.

# Список унікальних intent-ів
labels = list(set(item["intent"] for item in data))

# Створення моделі spaCy
nlp = spacy.blank("en")

# Додаємо TextCategorizer до конвеєра та налаштовуємо через конфіг
textcat = nlp.add_pipe("textcat")
textcat.cfg["exclusive_classes"] = True # тільки одна категорія може бути активною (один intent), а інші по нулям
textcat.cfg["architecture"] = "bow"  # сумка слів або "simple_cnn" (простий згортковий класифікатор)

for label in labels:
    textcat.add_label(label)


train_examples = [Example.from_dict(nlp.make_doc(text),label) for text,label in train_data] # Приклади для ініціалізації параметрів (ваг) моделі з пари текст-анотація
# Ініціалізація параметрів компоненту textcat
#textcat.initialize(lambda: train_examples, nlp=nlp)

# Ініціалізація параметрів моделі
optimizer = nlp.initialize(lambda: train_examples) # spaCy використовує оптимізатор Adam (Оновлює ваги на основі середнього значення градієнтів (momentum) та середньо-квадратичного значення градієнтів)

# Навчання моделі
for epoch in range(15):
    random.shuffle(train_data)
    losses = {}
    batches = minibatch(train_data, size=4)
    for batch in batches:
        examples = []
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            examples.append(Example.from_dict(doc, annotations))
        nlp.update(examples, sgd=optimizer, losses=losses)
    print(f"Epoch {epoch + 1}, Loss: {losses['textcat']:.4f}")

# Тестування
test_texts = [
    "I need a technician to fix my internet.",
    "Can I get a cheaper plan?",
    "Cancel my internet service now.",
    "Tell me about your internet plans"
]

for text in test_texts:
    doc = nlp(text)
    predicted = max(doc.cats, key=doc.cats.get) # Вибираємо категорію з найбільшою ймовірністю
    print(f"\nText: {text}")
    print(f"Predicted intent: {predicted} (score: {doc.cats[predicted]:.3f})")


Epoch 1, Loss: 0.2189
Epoch 2, Loss: 0.2133
Epoch 3, Loss: 0.2001
Epoch 4, Loss: 0.1860
Epoch 5, Loss: 0.1667
Epoch 6, Loss: 0.1431
Epoch 7, Loss: 0.1248
Epoch 8, Loss: 0.0857
Epoch 9, Loss: 0.0701
Epoch 10, Loss: 0.0496
Epoch 11, Loss: 0.0338
Epoch 12, Loss: 0.0353
Epoch 13, Loss: 0.0221
Epoch 14, Loss: 0.0180
Epoch 15, Loss: 0.0108

Text: I need a technician to fix my internet.
Predicted intent: technician_request (score: 0.302)

Text: Can I get a cheaper plan?
Predicted intent: billing_question (score: 0.835)

Text: Cancel my internet service now.
Predicted intent: cancel_service (score: 0.833)

Text: Tell me about your internet plans
Predicted intent: pricing_info (score: 0.326)


In [ ]:
import json
import random
import spacy
from spacy.training.example import Example
from spacy.util import minibatch
from pathlib import Path

# Завантаження даних
with open("2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Підготовка тренувальних прикладів
train_data = []
for item in data:
    utterance = item["utterance"]  # Висловлювання (текст/повідомлення)
    intent = item["intent"]  # Намір повідомлення / категорія (клас)
    train_data.append((utterance, {"cats": {intent: 1.0}}))  # Cтворюємо тренувальний словник категорій для класифікації

# Список унікальних intent-ів
labels = list(set(item["intent"] for item in data))

# Створення моделі spaCy
nlp = spacy.blank("en")

# Додаємо TextCategorizer до конвеєра
textcat = nlp.add_pipe("textcat")
textcat.cfg["exclusive_classes"] = True  # тільки одна категорія може бути активною
textcat.cfg["architecture"] = "simple_cnn"  # сумка слів або "simple_cnn"

# Додаємо категорії до текстового класифікатора
for label in labels:
    textcat.add_label(label)

# Підготовка тренувальних прикладів
train_examples = [Example.from_dict(nlp.make_doc(text), annotations) for text, annotations in train_data]

# Ініціалізація параметрів компоненту textcat
textcat.initialize(lambda: train_examples, nlp=nlp)

# Ініціалізація оптимізатора (розпочинаємо навчання)
optimizer = nlp.begin_training()

# Навчання моделі
for epoch in range(15):
    random.shuffle(train_data)
    losses = {}
    batches = minibatch(train_data, size=4)
    for batch in batches:
        examples = []
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            examples.append(Example.from_dict(doc, annotations))
        # Оновлення ваг
        nlp.update(examples, sgd=optimizer, losses=losses)
    print(f"Epoch {epoch + 1}, Loss: {losses['textcat']:.4f}")

# Тестування моделі
test_texts = [
    "I need a technician to fix my internet.",
    "Can I get a cheaper plan?",
    "Cancel my internet service now.",
    "Tell me about your internet plans"
]

for text in test_texts:
    doc = nlp(text)
    predicted = max(doc.cats, key=doc.cats.get)  # Вибираємо категорію з найбільшою ймовірністю
    print(f"\nText: {text}")
    print(f"Predicted intent: {predicted} (score: {doc.cats[predicted]:.3f})")


Epoch 1, Loss: 0.3284
Epoch 2, Loss: 0.3144
Epoch 3, Loss: 0.2940
Epoch 4, Loss: 0.2611
Epoch 5, Loss: 0.2216
Epoch 6, Loss: 0.1737
Epoch 7, Loss: 0.1279
Epoch 8, Loss: 0.0853
Epoch 9, Loss: 0.0507
Epoch 10, Loss: 0.0269
Epoch 11, Loss: 0.0146
Epoch 12, Loss: 0.0076
Epoch 13, Loss: 0.0029
Epoch 14, Loss: 0.0011
Epoch 15, Loss: 0.0005

Text: I need a technician to fix my internet.
Predicted intent: technical_issue (score: 0.255)

Text: Can I get a cheaper plan?
Predicted intent: billing_question (score: 0.928)

Text: Cancel my internet service now.
Predicted intent: cancel_service (score: 0.511)

Text: Tell me about your internet plans
Predicted intent: pricing_info (score: 0.527)
